In [44]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")
import pandas as pd
import tensorflow.keras as keras
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import tensorflow as tf

In [45]:
data = np.load('data+meta.npz',allow_pickle = True)

meta = np.array(data['meta'])
# ["campanian","corylus","dust","grimsvotn","qrobur","qsuber"]

label = np.array(data['label'])
data = np.array(data['data'])

In [46]:
print(meta.shape)
print(data.shape)
print(label.shape)

(12000, 39)
(12000, 256, 256)
(12000,)


In [47]:
def standardizeimg(img, mu, sigma):
    return (img-mu)/(sigma).astype(np.float32)
data =  standardizeimg(img=data, mu=np.mean(data), sigma=np.std(data))
meta =  standardizeimg(img=meta, mu=np.mean(meta), sigma=np.std(meta))

In [48]:
data = data.reshape(data.shape[0], data.shape[1],data.shape[2], 1)

In [49]:
data1 = [data,meta]
print(data1[0].shape)
print(data1[1].shape)

(12000, 256, 256, 1)
(12000, 39)


In [50]:
NoP = 1000

sample = np.random.randint(0,len(data1[1][:,0]),NoP)

data_small = [data1[0][sample],data1[1][sample]]
label_small = label[sample]
print(data_small[0].shape)
print(data_small[1].shape)

(1000, 256, 256, 1)
(1000, 39)


In [51]:
model_3 = keras.models.load_model("model_3type")
model_ash = keras.models.load_model("model_ash") # camp og grim , [0,3]
model_pollen = keras.models.load_model("model_pollen")

In [52]:
prediction_p = model_3.predict(data_small)

In [56]:
predictions = np.argmax(prediction_p, axis=1)

predicted = np.zeros(len(predictions))


print(predictions)


pollen = [data_small[0][predictions == 1],data_small[1][predictions == 1]]
label_pollen = label_small[predictions == 1]

ash = [data_small[0][predictions == 0],data_small[1][predictions == 0]]
label_ash = label_small[predictions == 0]

dust = [data_small[0][predictions == 2],data_small[1][predictions == 2]]
predicted[predictions == 2] = 2





[1 0 0 2 0 0 1 1 0 1 0 1 2 2 1 0 0 2 2 1 1 0 0 0 1 1 2 1 1 1 2 0 1 0 0 2 2
 2 0 0 1 1 2 0 0 0 0 0 0 1 0 1 1 0 2 1 2 1 1 1 1 1 1 1 1 1 1 1 2 0 0 1 1 0
 1 0 0 1 1 0 1 1 2 0 1 1 2 0 2 0 0 2 1 1 1 0 0 0 2 0 1 2 0 2 1 0 2 1 1 0 2
 0 1 1 0 2 1 2 1 2 2 1 0 2 1 2 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0 1 0 2 0
 1 0 2 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 2 0 1 1 0 2 0 0
 1 1 0 0 1 1 0 0 0 1 1 0 1 0 1 1 2 0 1 1 0 2 1 1 0 1 2 1 1 1 1 1 0 0 1 0 2
 2 0 1 1 2 0 2 1 1 1 1 2 1 1 1 0 0 0 2 1 0 2 2 1 1 1 1 0 1 0 1 0 2 0 0 0 1
 1 1 2 1 1 0 1 1 1 1 2 0 0 1 1 2 1 1 1 2 0 1 0 1 0 0 0 1 1 1 2 1 0 2 2 1 1
 0 1 1 1 2 1 0 1 1 1 1 0 1 0 1 1 1 0 0 2 1 0 0 0 1 1 0 0 1 1 1 1 0 0 2 0 1
 1 2 1 1 1 1 2 1 2 0 0 0 0 1 1 0 1 0 2 2 1 1 0 1 1 2 1 1 2 1 2 0 0 1 1 1 2
 0 2 2 1 1 1 1 1 0 1 1 0 2 0 1 1 1 0 1 0 1 1 0 0 1 0 1 2 1 1 1 0 1 0 0 0 0
 1 1 2 2 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 2 1 1 0 1 1 0 2 0 2 0
 2 1 0 0 1 2 1 2 0 0 1 1 0 2 1 1 1 0 1 1 0 2 0 1 1 2 0 1 2 1 1 2 1 2 1 1 1
 0 1 0 0 1 1 2 2 2 2 1 0 

In [57]:
print(label_dust)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0
 2 2 2 2 0 2 2 2 2 3 0 2 2 2 0 2 2 2 2 2 3 2 2 2 2 0 3 2 2 2 0 2 2 2 2 2 2
 2 2 3 2 3 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 3 2 2 2 2 2 2 2 2 3 2
 2 2 0 2 2 2 0 2 2 2 2 3 2 2 0 2 3 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2
 2 2 3 0 2 3 2 2 2 2 0 2 2]


In [58]:
pollen_p = model_pollen(pollen)
ash_p = model_ash(ash)